<a href="https://colab.research.google.com/github/poorvigupta26/LEETCODE_JAVA/blob/main/nlp_to_fn_pipeline_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install -U langchain langchain-community huggingface_hub transformers

In [47]:
import json
import os
import re
from datetime import datetime

In [48]:
function_library = [
    {
        "name": "get_invoices",
        "description": "Fetches all invoice records for a given month.",
        "inputs": {
            "month": "string"
        },
        "outputs": {
            "invoices": "list"
        }
    },
    {
        "name": "summarize_invoices",
        "description": "Summarizes a list of invoices to get total amount and stats.",
        "inputs": {
            "invoices": "list"
        },
        "outputs": {
            "summary": "string"
        }
    },
    {
        "name": "send_email",
        "description": "Sends an email with subject and body to a recipient.",
        "inputs": {
            "to": "string",
            "subject": "string",
            "body": "string"
        },
        "outputs": {
            "status": "string"
        }
    },
    {
        "name": "get_meetings",
        "description": "Retrieves scheduled meetings for a specified date.",
        "inputs": {
            "date": "string"
        },
        "outputs": {
            "meetings": "list"
        }
    },
    {
        "name": "schedule_meeting",
        "description": "Schedules a new meeting.",
        "inputs": {
            "participants": "list",
            "date": "string",
            "time": "string",
            "agenda": "string"
        },
        "outputs": {
            "confirmation": "string"
        }
    },
    {
        "name": "get_user_profile",
        "description": "Fetches profile data for a given user ID.",
        "inputs": {
            "user_id": "string"
        },
        "outputs": {
            "profile": "dict"
        }
    },
    {
        "name": "update_user_preferences",
        "description": "Updates a user's notification or theme preferences.",
        "inputs": {
            "user_id": "string",
            "preferences": "dict"
        },
        "outputs": {
            "status": "string"
        }
    },
    {
        "name": "fetch_weather",
        "description": "Gets weather information for a specified location.",
        "inputs": {
            "location": "string"
        },
        "outputs": {
            "weather": "dict"
        }
    },
    {
        "name": "log_activity",
        "description": "Logs a user activity for audit or tracking purposes.",
        "inputs": {
            "user_id": "string",
            "activity": "string",
            "timestamp": "string"
        },
        "outputs": {
            "log_id": "string"
        }
    },
    {
        "name": "translate_text",
        "description": "Translates input text from one language to another.",
        "inputs": {
            "text": "string",
            "source_lang": "string",
            "target_lang": "string"
        },
        "outputs": {
            "translated_text": "string"
        }
    },
    {
        "name": "fetch_news_headlines",
        "description": "Retrieves top news headlines for a given category or location.",
        "inputs": {
            "category": "string",
            "region": "string"
        },
        "outputs": {
            "headlines": "list"
        }
    },
    {
        "name": "create_task",
        "description": "Creates a to-do task for the user.",
        "inputs": {
            "title": "string",
            "due_date": "string",
            "priority": "string"
        },
        "outputs": {
            "task_id": "string"
        }
    }
]


with open("function_library.json", "w") as f:
    json.dump(function_library, f, indent=2)

print("function_library created")


function_library created


In [49]:
def format_function_prompt(function_registry):
    prompt = ""
    for fn in function_registry:
        name = fn["name"]
        desc = fn["description"]
        inputs = fn["inputs"]
        inputs_str = ", ".join(f"{k}: {v}" for k, v in inputs.items())
        prompt += f"- {name}({inputs_str}): {desc}\n"
    return prompt


print(format_function_prompt(function_library));

- get_invoices(month: string): Fetches all invoice records for a given month.
- summarize_invoices(invoices: list): Summarizes a list of invoices to get total amount and stats.
- send_email(to: string, subject: string, body: string): Sends an email with subject and body to a recipient.
- get_meetings(date: string): Retrieves scheduled meetings for a specified date.
- schedule_meeting(participants: list, date: string, time: string, agenda: string): Schedules a new meeting.
- get_user_profile(user_id: string): Fetches profile data for a given user ID.
- update_user_preferences(user_id: string, preferences: dict): Updates a user's notification or theme preferences.
- fetch_weather(location: string): Gets weather information for a specified location.
- log_activity(user_id: string, activity: string, timestamp: string): Logs a user activity for audit or tracking purposes.
- translate_text(text: string, source_lang: string, target_lang: string): Translates input text from one language to ano

In [50]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

cleaning_prompt = PromptTemplate.from_template("""
You are a query normalizer assistant. Rewrite the following user query into a clean, professional, and structured instruction that can be easily interpreted by an AI planner.

Original Query: "{query}"
Cleaned Instruction:
""")


In [51]:
import google.generativeai as genai

In [52]:
import os
from google.colab import userdata

# os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['google_genai_key'] = userdata.get('GOOGLE_API_KEY')

In [53]:
genai.configure(api_key=os.environ["google_genai_key"])

In [54]:
from langchain.schema.runnable import RunnableLambda

# Generic Gemini wrapper for any prompt
def gemini_wrapper(prompt: str) -> str:
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text.strip()

In [55]:
# Gemini-based prompt template
def clean_with_gemini(raw_query):
    prompt = f"""
You are a query normalizer. Rewrite vague or informal user queries into clear, structured instructions.

Original: "{raw_query}"
Cleaned:"""
    return gemini_wrapper(prompt)

# Wrap as LangChain Runnable
gemini_cleaner = RunnableLambda(lambda x: {"user_query": clean_with_gemini(x["query"])})

In [56]:
def plan_with_gemini(cleaned_query, function_library):
    prompt = f"""
You are a function planning assistant. Given a structured task, output the ordered list of function calls needed.

Available functions:
{format_function_prompt(function_library)}

Task: {cleaned_query}

Plan:
"""
    return gemini_wrapper(prompt)

gemini_planner = RunnableLambda(lambda x: {"plan": plan_with_gemini(x["user_query"], function_library)})

In [57]:
pipeline = gemini_cleaner | gemini_planner


In [58]:
def run_pipeline(query):
     return pipeline.invoke({"query": query})["plan"]


In [59]:
from IPython.display import display
import ipywidgets as widgets

In [60]:
# Input box
input_box = widgets.Text(
    value='grab my bills from may 2024',
    placeholder='Enter your query here',
    description='Query:',
    layout=widgets.Layout(width='100%')
)

# Run button
run_button = widgets.Button(
    description='Generate Plan',
    button_style='success'
)

# Output area
output_area = widgets.Output()


In [61]:
def on_run_button_clicked(b):
    output_area.clear_output()
    with output_area:
        print("⏳ Generating plan... please wait.")
    with output_area:
        user_input = input_box.value
        try:
            result = run_pipeline(user_input)
            print("Function Plan:\n", result)
        except Exception as e:
            print("Error:", str(e))

run_button.on_click(on_run_button_clicked)


In [69]:
display(input_box, run_button, output_area)


Text(value='tell me abt the bills from last month and schedule a meeting with david tonight to dicsuss about t…

Button(button_style='success', description='Generate Plan', style=ButtonStyle())

Output()

Function Plan:
 1. **Get invoices from the previous month:** `get_invoices(month)` needs to be called, where `month` should be the previous month (e.g., "October" if it is currently November).
2. **Summarize the invoices:** `summarize_invoices(invoices)` should be called using the output from step 1.
3. **Schedule a meeting with David:** `schedule_meeting(participants, date, time, agenda)` needs to be called, where `participants` includes "David", `date` is "today", `time` should be a suitable time "tonight", and `agenda` should relate to discussing the bills.

Output:
```
1. get_invoices(month="previous month")
2. summarize_invoices(invoices="result of get_invoices")
3. schedule_meeting(participants=["David"], date="today", time="tonight", agenda="Discuss summarized invoices from previous month")
```
